In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data Model - Pizza Sales.xlsx to Data Model - Pizza Sales.xlsx


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Use simple local folders (inside Colab environment)
DATA_RAW = Path("/content/raw")
DATA_INTERIM = Path("/content/interim")
DATA_OUTPUTS = Path("/content/outputs")

# Create the folders
for folder in [DATA_RAW, DATA_INTERIM, DATA_OUTPUTS]:
    folder.mkdir(exist_ok=True)

print("✅ Folders ready in Colab!")


✅ Folders ready in Colab!


In [ ]:
import shutil

# Move uploaded Excel to /content/raw/
shutil.move("Data Model - Pizza Sales.xlsx", DATA_RAW / "Data Model - Pizza Sales.xlsx")
print("✅ File moved to raw folder!")


✅ File moved to raw folder!


In [ ]:
# ---- Step 3: Load the pizza sales Excel file ----

excel_path = DATA_RAW / "Data Model - Pizza Sales.xlsx"  # path to Excel file

# Read Excel file using pandas
df = pd.read_excel(excel_path, engine="openpyxl")

# Display first few rows
df.head()


,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [ ]:
# ---- Step 4: Clean column names ----

df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

print("Cleaned column names:")
print(df.columns.tolist())


Cleaned column names:
['order_details_id', 'order_id', 'pizza_id', 'quantity', 'order_date', 'order_time', 'unit_price', 'total_price', 'pizza_size', 'pizza_category', 'pizza_ingredients', 'pizza_name']


In [ ]:
# ---- Step 5: Inspect the dataset ----

print("Data types:")
print(df.dtypes)
print("\nMissing values per column:")
print(df.isnull().sum())


Data types:
order_details_id              int64
order_id                      int64
pizza_id                     object
quantity                      int64
order_date           datetime64[ns]
order_time                   object
unit_price                  float64
total_price                 float64
pizza_size                   object
pizza_category               object
pizza_ingredients            object
pizza_name                   object
dtype: object

Missing values per column:
order_details_id     0
order_id             0
pizza_id             0
quantity             0
order_date           0
order_time           0
unit_price           0
total_price          0
pizza_size           0
pizza_category       0
pizza_ingredients    0
pizza_name           0
dtype: int64


In [ ]:
# ---- Step 6: Convert order_date to datetime and extract useful time features ----

df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')  # convert to date format

# Extract useful parts
df['order_day'] = df['order_date'].dt.day_name()   # Monday, Tuesday, etc.
df['order_month'] = df['order_date'].dt.month_name()
df['order_hour'] = df['order_date'].dt.hour        # numeric hour (if time included)

df[['order_date', 'order_day', 'order_month', 'order_hour']].head()


,order_date,order_day,order_month,order_hour
0,2015-01-01,Thursday,January,0
1,2015-01-01,Thursday,January,0
2,2015-01-01,Thursday,January,0
3,2015-01-01,Thursday,January,0
4,2015-01-01,Thursday,January,0


In [ ]:
# ---- Step 7: Ensure numeric columns and calculate total_price ----

# Convert text to numbers safely
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').fillna(0).astype(int)
df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce').fillna(0.0)

# Create total_price column
df['total_price'] = df['quantity'] * df['unit_price']

df[['pizza_name', 'quantity', 'unit_price', 'total_price']].head()


,pizza_name,quantity,unit_price,total_price
0,The Hawaiian Pizza,1,13.25,13.25
1,The Classic Deluxe Pizza,1,16.00,16.00
2,The Five Cheese Pizza,1,18.50,18.50
3,The Italian Supreme Pizza,1,20.75,20.75
4,The Mexicana Pizza,1,16.00,16.00


In [ ]:
# ---- Step 8: Save cleaned dataset ----

cleaned_path = DATA_INTERIM / "pizza_sales_clean.csv"
df.to_csv(cleaned_path, index=False)

print(f"✅ Cleaned data saved successfully to: {cleaned_path}")


✅ Cleaned data saved successfully to: /content/interim/pizza_sales_clean.csv


In [ ]:
# ---- Step 9: Quick summary ----
print("Total Orders:", df['order_id'].nunique())
print("Total Revenue:", round(df['total_price'].sum(), 2))
print("Top 5 pizzas by quantity sold:")
print(df.groupby('pizza_name')['quantity'].sum().sort_values(ascending=False).head())


Total Orders: 21350
Total Revenue: 817860.05
Top 5 pizzas by quantity sold:
pizza_name
The Classic Deluxe Pizza      2453
The Barbecue Chicken Pizza    2432
The Hawaiian Pizza            2422
The Pepperoni Pizza           2418
The Thai Chicken Pizza        2371
Name: quantity, dtype: int64


In [ ]:
from google.colab import files
files.download(DATA_INTERIM / "pizza_sales_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>